In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision.models import resnext50_32x4d
from torch.utils.data import DataLoader

In [2]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to the input size of the model
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [3]:
train_data = ImageFolder(root='train', transform=transform)
val_data = ImageFolder(root='validation', transform=transform)
test_data = ImageFolder(root='test', transform=transform)

# Creating data loaders
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [4]:
# Second transformation to try later.
# from torchvision.transforms import Resize, Pad, ToTensor, Normalize, Compose, CenterCrop

# def transform_with_padding(image_width=436, image_height=184, target_size=224):
#     # Calculate padding
#     aspect_ratio = image_width / image_height
#     if aspect_ratio > 1:
#         new_height = target_size
#         new_width = int(aspect_ratio * new_height)
#         padding = (new_width - target_size) // 2
#         padding = (padding, 0, padding, 0)  # left, top, right, bottom
#     else:
#         new_width = target_size
#         new_height = int(new_width / aspect_ratio)
#         padding = (0, (new_height - target_size) // 2, 0, (new_height - target_size) // 2)

#     return Compose([
#         Resize((new_height, new_width)),
#         Pad(padding, fill=0, padding_mode='constant'),
#         ToTensor(),
#         Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
#     ])

# transform = transform_with_padding()

# # Apply this transform to your datasets
# train_data = ImageFolder(root='train', transform=transform)
# val_data = ImageFolder(root='validation', transform=transform)
# test_data = ImageFolder(root='test', transform=transform)

In [5]:
print(train_data)

Dataset ImageFolder
    Number of datapoints: 7156
    Root location: train
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )


In [6]:
print(train_data[1])

(tensor([[[2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
         [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
         [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
         ...,
         [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
         [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489],
         [2.2489, 2.2489, 2.2489,  ..., 2.2489, 2.2489, 2.2489]],

        [[2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
         [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
         [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
         ...,
         [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
         [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286],
         [2.4286, 2.4286, 2.4286,  ..., 2.4286, 2.4286, 2.4286]],

        [[2.6400, 2.6400, 2.6400,  ..., 2.6400, 2.6400, 2.6400],
         [2.6400, 2.6400, 2.6400,  ..., 2.6400, 2.6400, 2.6400],
         [2.6400, 2.6400, 2.6400,  ..., 2.6400, 2.6400, 2

In [7]:
model = resnext50_32x4d(pretrained=True)

C:\Users\HP\anaconda3\envs\signature_verification\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\HP\anaconda3\envs\signature_verification\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt50_32X4D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
num_features = model.fc.in_features
model.fc = torch.nn.Linear(num_features, 2)

In [9]:
print(num_features)

2048


In [10]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [11]:
from tqdm import tqdm
import torch.nn.functional as F

In [12]:
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train() 

    for epoch in range(num_epochs):
        running_loss = 0.0
        correct_predictions = 0

        for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}', unit='batch'):
            optimizer.zero_grad()  # Zero the parameter gradients

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            # Statistics
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = correct_predictions / len(train_loader.dataset)

        print(f'Epoch {epoch + 1}: Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')


In [13]:
model = resnext50_32x4d(pretrained=True)
num_features = model.fc.in_features
model.fc = torch.nn.Linear(num_features, 2)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [14]:
num_epochs = 5  # Adjust 
train_model(model, train_loader, criterion, optimizer, num_epochs=num_epochs)

Epoch 1/5: 100%|███████████████████████████████████████████████████████████████████| 224/224 [1:09:48<00:00, 18.70s/batch]


Epoch 1: Loss: 0.3841, Accuracy: 0.8407


Epoch 2/5: 100%|███████████████████████████████████████████████████████████████████| 224/224 [1:05:37<00:00, 17.58s/batch]


Epoch 2: Loss: 0.2745, Accuracy: 0.8840


Epoch 3/5: 100%|███████████████████████████████████████████████████████████████████| 224/224 [1:04:04<00:00, 17.16s/batch]


Epoch 3: Loss: 0.2260, Accuracy: 0.9053


Epoch 4/5: 100%|███████████████████████████████████████████████████████████████████| 224/224 [1:03:42<00:00, 17.06s/batch]


Epoch 4: Loss: 0.1798, Accuracy: 0.9248


Epoch 5/5: 100%|███████████████████████████████████████████████████████████████████| 224/224 [1:09:30<00:00, 18.62s/batch]

Epoch 5: Loss: 0.1696, Accuracy: 0.9307


In [15]:
# Save the model's state dictionary in the root directory
model_save_path = 'resnext50_model_state.pth' 
torch.save(model.state_dict(), model_save_path)
print(f'Model state dictionary saved to {model_save_path}')

Model state dictionary saved to resnext50_model_state.pth


In [16]:
# Load the saved state dictionary
model.load_state_dict(torch.load(model_save_path))
print('Model loaded successfully.')

Model loaded successfully.


In [17]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

In [19]:
from torch.nn.functional import softmax


In [20]:
from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve
import numpy as np

def evaluate_model(model, data_loader):
    model.eval()  # Set the model to evaluation mode

    all_scores = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in tqdm(data_loader, desc='Evaluating', unit='batch'):
            outputs = model(inputs)
            probabilities = softmax(outputs, dim=1)[:, 1]
            all_scores.extend(probabilities.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Convert scores to binary predictions using a threshold (e.g., 0.5)
    threshold = 0.5
    predictions = [1 if score > threshold else 0 for score in all_scores]

    # Confusion Matrix
    tn, fp, fn, tp = confusion_matrix(all_labels, predictions).ravel()

    # Calculate Accuracy
    acc = accuracy_score(all_labels, predictions)

    # Calculate EER
    fpr, tpr, thresholds = roc_curve(all_labels, all_scores)
    fnr = 1 - tpr
    eer_threshold = thresholds[np.nanargmin(np.absolute((fnr - fpr)))]
    eer = fpr[np.nanargmin(np.absolute((fnr - fpr)))]

    # Additional Calculations
    num_false_accepted = fp  # Forged signatures incorrectly classified as genuine
    num_false_rejected = fn  # Genuine signatures incorrectly classified as forgeries
    total_num_forged = fp + tn  # Total number of forged signatures
    total_num_genuine = fn + tp  # Total number of genuine signatures

    return acc, eer, num_false_accepted, num_false_rejected, total_num_forged, total_num_genuine

# Evaluate on Validation and Test Sets
val_acc, val_eer, val_fa, val_fr, val_total_forged, val_total_genuine = evaluate_model(model, val_loader)
print(f'Validation: Accuracy: {val_acc:.4f}, EER: {val_eer:.4f}, False Accepted: {val_fa}, False Rejected: {val_fr}, Total Forged: {val_total_forged}, Total Genuine: {val_total_genuine}')

test_acc, test_eer, test_fa, test_fr, test_total_forged, test_total_genuine = evaluate_model(model, test_loader)
print(f'Test: Accuracy: {test_acc:.4f}, EER: {test_eer:.4f}, False Accepted: {test_fa}, False Rejected: {test_fr}, Total Forged: {test_total_forged}, Total Genuine: {test_total_genuine}')

Evaluating: 100%|██████████████████████████████████████████████████████████████████████| 47/47 [04:07<00:00,  5.27s/batch]


Validation: Accuracy: 0.6188, EER: 0.4491, False Accepted: 336, False Rejected: 232, Total Forged: 1091, Total Genuine: 399


Evaluating: 100%|██████████████████████████████████████████████████████████████████████| 50/50 [04:27<00:00,  5.35s/batch]

Test: Accuracy: 0.5909, EER: 0.3948, False Accepted: 522, False Rejected: 131, Total Forged: 1198, Total Genuine: 398


In [21]:
FPR = 131/398
FAR = 522/1198
TAR = 1 - FPR
print(f'FAR: {FAR}, FPR: {FPR}, TAR: {TAR}')

FAR: 0.4357262103505843, FPR: 0.32914572864321606, TAR: 0.670854271356784


In [ ]:
print(train_data)

In [ ]:
print(test_data)

In [ ]:
print(val_data)